In [1]:
from PIL import Image
import os

image_dir = 'EuropeanCastles/Castle_Images'

# Walk through the directories and subdirectories
for subdir, dirs, files in os.walk(image_dir):
    for file in files:
        try:
            img = Image.open(os.path.join(subdir, file)) # open the image file
            img.verify() # verify that it is, in fact an image
        except (IOError, SyntaxError) as e:
            print('Bad file:', os.path.join(subdir, file)) # print out the names of corrupt files
print("no bad files")

no bad files


In [2]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"
import PIL
from tensorflow import keras
from tensorflow.keras.preprocessing import image
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [3]:
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

print("training set: ")
train_generator = datagen.flow_from_directory(
    'EuropeanCastles/Castle_Images',
    target_size = (480,480),
    batch_size = 32,
    class_mode = 'categorical',
    subset = 'training'
)

print("validation set: ")
validation_generator = datagen.flow_from_directory(
    'EuropeanCastles/Castle_Images',
    target_size = (480,480),
    batch_size = 32,
    class_mode = 'categorical',
    subset = 'validation'
)

training set: 
Found 4268 images belonging to 37 classes.
validation set: 
Found 1057 images belonging to 37 classes.


In [4]:
num_classes = 37

model = Sequential()
model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

c:\Users\VamsiKarnam\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [5]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [6]:
history = model.fit(
    train_generator,
    steps_per_epoch = None, # train_generator.samples // train_generator.batch_size,
    validation_data = validation_generator,
    validation_steps = None, epochs = 7) # validation_generator.samples // validation_generator.batch_size, epochs=20)

Epoch 1/7


c:\Users\VamsiKarnam\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


134/134 ━━━━━━━━━━━━━━━━━━━━ 113s 825ms/step - accuracy: 0.1409 - loss: 3.3960 - val_accuracy: 0.1854 - val_loss: 2.8662
Epoch 2/7
134/134 ━━━━━━━━━━━━━━━━━━━━ 111s 823ms/step - accuracy: 0.1840 - loss: 2.8561 - val_accuracy: 0.1854 - val_loss: 2.8556
Epoch 3/7
134/134 ━━━━━━━━━━━━━━━━━━━━ 110s 812ms/step - accuracy: 0.1831 - loss: 2.8870 - val_accuracy: 0.1854 - val_loss: 2.8667
Epoch 4/7
134/134 ━━━━━━━━━━━━━━━━━━━━ 111s 825ms/step - accuracy: 0.1830 - loss: 2.9075 - val_accuracy: 0.1854 - val_loss: 2.8599
Epoch 5/7
134/134 ━━━━━━━━━━━━━━━━━━━━ 110s 816ms/step - accuracy: 0.1889 - loss: 2.8549 - val_accuracy: 0.1816 - val_loss: 2.8925
Epoch 6/7
134/134 ━━━━━━━━━━━━━━━━━━━━ 111s 820ms/step - accuracy: 0.2310 - loss: 2.6430 - val_accuracy: 0.1504 - val_loss: 3.0765
Epoch 7/7
134/134 ━━━━━━━━━━━━━━━━━━━━ 111s 822ms/step - accuracy: 0.3456 - loss: 2.2015 - val_accuracy: 0.1429 - val_loss: 3.3923


In [7]:
loss, accuracy = model.evaluate(validation_generator)
print(f'Validation accuracy : {accuracy}')

34/34 ━━━━━━━━━━━━━━━━━━━━ 7s 201ms/step - accuracy: 0.1254 - loss: 3.4545
Validation accuracy : 0.1428571492433548


In [8]:
img = image.load_img('EuropeanCastles/Castle_Images/Isle_of_Man/Peel_Castle/1.jpg', target_size=(480,480))

img_array = image.img_to_array(img)
img_array = img_array / 255.0

img_array = np.expand_dims(img_array, axis=0)

prediction = model.predict(img_array)

predicted_class = np.argmax(prediction[0])

class_indices = train_generator.class_indices

index_to_class = {v: k for k, v in class_indices.items()}

predicted_class_name = index_to_class[predicted_class]

print('Predicted class: ', predicted_class_name)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
Predicted class:  United_Kingdom


In [9]:
# Save model and labels

import pickle

with open('class_indices.pkl', 'wb') as f:
    pickle.dump(train_generator.class_indices, f)

model.save('locate.h5') # h5 is legacy, if encoutering errors, save as .keras instead

In [10]:
# load model and predict example
"""
from tensorflow.keras.models import load_model
import pickle

with open('class_indices.pkl', 'wb') as f:
    class_indices = pickle.load(f)

model = load_model('path_to_h5_file.h5')

img = image.load_img('EuropeanCastles/Castle_Images/Isle_of_Man/Peel_Castle/1.jpg', target_size=(150,150))

img_array = image.img_to_array(img)
img_array = img_array / 255.0

img_array = np.expand_dims(img_array, axis=0)

prediction = model.predict(img_array)

predicted_class = np.argmax(prediction[0])

index_to_class = {v: k for k, v in class_indices.items()}

predicted_class_name = index_to_class[predicted_class]

print('Predicted class: ', predicted_class_name) 

"""

"\nfrom tensorflow.keras.models import load_model\nimport pickle\n\nwith open('class_indices.pkl', 'wb') as f:\n    class_indices = pickle.load(f)\n\nmodel = load_model('path_to_h5_file.h5')\n\nimg = image.load_img('EuropeanCastles/Castle_Images/Isle_of_Man/Peel_Castle/1.jpg', target_size=(150,150))\n\nimg_array = image.img_to_array(img)\nimg_array = img_array / 255.0\n\nimg_array = np.expand_dims(img_array, axis=0)\n\nprediction = model.predict(img_array)\n\npredicted_class = np.argmax(prediction[0])\n\nindex_to_class = {v: k for k, v in class_indices.items()}\n\npredicted_class_name = index_to_class[predicted_class]\n\nprint('Predicted class: ', predicted_class_name) \n\n"